In [5]:
# ========== ----- ========== Import Libraries ========== ----- ========== #

from joblib import dump, load
from collections import Counter
import pickle
from LDP import LDP_TOP
from LDP import train
import sklearn
print(sklearn.__version__)

import numpy as np
import dlib
import cv2
import os
import re
import json
import numpy as np
import pandas as pd

from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# ========== ----- ========== End ========== ----- ========== #

1.2.2


In [6]:
# ========== ----- ========== Reading Training Data ========== ----- ========== #

# Reading Training Data
train_frame_folders = [
    'faceforensics_dataset_train/original_sequences',
    # 'faceforensics_dataset_train/manipulated_sequences/Deepfakes',
    # 'faceforensics_dataset_train/manipulated_sequences/Face2Face',
    # 'faceforensics_dataset_train/manipulated_sequences/FaceSwap',
    # 'faceforensics_dataset_train/manipulated_sequences/NeuralTextures'
]

# ========== ----- ========== End ========== ----- ========== #

In [ ]:
# ========== ----- ========== Feature Extraction ========== ----- ========== #

# Face detector
detector = dlib.get_frontal_face_detector()

list_of_train_LDP = []
list_of_train_labels = []

for train_frame_folder in train_frame_folders:
    list_of_train_data = [f for f in os.listdir(
        train_frame_folder) if f.endswith('.mp4')]
    print(train_frame_folder, 'videos: ', len(list_of_train_data))
    for vid in list_of_train_data:
        frames = []
        # Video capturing constructor.
        cap = cv2.VideoCapture(os.path.join(train_frame_folder, vid))

        # (CAP_PROP_FPS) Returns frame rate of the video (#frames / second).
        frameRate = cap.get(5)
        while cap.isOpened():  # Returns true if video capturing has been initialized already.

            # (CAP_PROP_POS_MSEC) Current position of the video file in milliseconds or video capture timestamp.
            frameId = cap.get(1)
            ret, frame = cap.read()  # The methods/functions combine VideoCapture::grab and VideoCapture::retrieve in one call. This is the most convenient method for reading video files or capturing data from decode and return the just grabbed frame. If no frames has been grabbed (camera has been disconnected, or there are no more frames in video file), the methods return false and the functions return NULL pointer.
            if ret != True:
                break

            face_rects, scores, idx = detector.run(frame, 0)
            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()

                crop_img = frame[y1:y2, x1:x2]
                if crop_img is not None and crop_img.size > 0:
                    crop_img = cv2.resize(crop_img, (128, 128))
                    gray_scale_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
                    frames.append(gray_scale_img)

            # d = 3 seconds.
            if frameId % ((int(frameRate)+1)*3) == 0:
                if (len(frames) > 0):
                    frames_ldp = LDP_TOP(np.array(frames).astype(np.float64))
                    list_of_train_LDP.append(frames_ldp)
                    if train_frame_folder == 'faceforensics_dataset_train/original_sequences':
                        list_of_train_labels.append(0)
                    else:
                        list_of_train_labels.append(1)
                frames = []

# ========== ----- ========== End ========== ----- ========== #


In [7]:
# ========== ----- ========== Load Features ========== ----- ========== #

# Loading the list_of_train_LDP
with open('features/list_of_train_LDP.pkl', 'rb') as f:
    list_of_train_LDP = pickle.load(f)

# Loading the list_of_train_labels
with open('features/list_of_train_labels.pkl', 'rb') as f:
    list_of_train_labels = pickle.load(f)

# ========== ----- ========== End ========== ----- ========== #

In [8]:
# ========== ----- ========== Prepare Features ========== ----- ========== #

counts = Counter(list_of_train_labels)

for value, count in counts.items():
    print(f"{value}: {count}")

# NeuralTextures
list_of_train_labels = list_of_train_labels[:5666] + \
    list_of_train_labels[21571:26141]

list_of_train_LDP = list_of_train_LDP[:5666] + \
    list_of_train_LDP[21571:26141]

counts = Counter(list_of_train_labels)

for value, count in counts.items():
    print(f"{value}: {count}")

list_of_train_labels = [0] * 5666 + [1] * 4570

counts = Counter(list_of_train_labels)

for value, count in counts.items():
    print(f"{value}: {count}")
    
# ========== ----- ========== End ========== ----- ========== #


0: 5666
1: 5666
2: 5669
3: 4570
4: 4570
0: 5666
4: 4570
0: 5666
1: 4570


In [10]:
# ========== ----- ========== Training SVM ========== ----- ========== #

model = svm.SVC(kernel='rbf')
model.fit(list_of_train_LDP, list_of_train_labels)
# Save the trained model to a file
dump(model, 'RBF-NeuralTextures.joblib')

# ================================================ #

model = svm.SVC(kernel='linear')
model.fit(list_of_train_LDP, list_of_train_labels)
# Save the trained model to a file
dump(model, 'Linear-NeuralTextures.joblib')

# ========== ----- ========== End ========== ----- ========== #

['Linear-NeuralTextures.joblib']